In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c house-prices-advanced-regression-techniques
!unzip house-prices-advanced-regression-techniques

mkdir: cannot create directory ‘/root/.kaggle’: File exists
house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
df_train = pd.read_csv('train.csv')
X, y = df_train.drop(['SalePrice', 'Id'], axis=1), df_train.SalePrice
df_test = pd.read_csv('test.csv')

###Класс для подготовки данных:

In [ ]:
class Data():
    """
    Класс для подготовки датасета к обучению/тестированию.
    Включает в себя генерацию признаков, заполнение пропусков.
    """

    def __init__(self, data: pd.DataFrame, is_train: bool=True):
        self.data = data.copy()
        self.is_train = is_train

        self.categorical = self.data.select_dtypes(exclude='number').columns
        self.numerical = self.data.select_dtypes(exclude='object').columns

        #ниже перечислены вещественные признаки,
        #которые будут переведены в бинарные
        self.zero_threshold       =   ['OpenPorchSF',
                                      'EnclosedPorch',
                                      '3SsnPorch',
                                      'ScreenPorch',
                                      'LowQualFinSF']

        self.one_threshold        =   'KitchenAbvGr'


        self.mlb = MultiLabelBinarizer()
        self.mlb.fit(df_train[['BsmtFinType1',
                               'BsmtFinType2',
                               'Exterior1st',
                               'Exterior2nd']]\
                .fillna('NO').to_numpy())


    def _QC_encoder(self, qual: str) -> int:
        """
        Метод для кодировки порядкого значения качества (str)
        в целое число.
        """

        qc = {
            'Po': 1,
            'Fa': 2,
            'TA': 3,
            'Gd': 4,
            'Ex': 5
        }

        if qual not in qc:
            return 0
        return qc[qual]


    def _encode_exposure(self, value: str) -> int:
        """
        Метод для кодировки порядкого значения (str) с 3 вариантами
        в целое число.
        """

        qc = {
              'Mn': 1,
              'Av': 2,
              'Gd': 3
          }

        if value not in qc:
            return 0
        return qc[value]


    def _encode_garage(self, value: str) -> int:
        """
        Метод для кодировки порядкого значения (str)
        степени завершенности гаража в целое число.
        """

        encoder = {
            'Unf': 1,
            'RFn': 2,
            'Fin': 3
        }

        if value not in encoder:
            return 0
        return encoder[value]


    def _multilabel(self):
        """
        Метод превращает признаки, которые могут содержать одновременно
        больше одной категории, в вектор из признаков, значения которых
        равны 0 или 1.
        """

        self.data[self.mlb.classes_] = \
        self.mlb.transform(
            self.data[['BsmtFinType1',
                       'BsmtFinType2',
                       'Exterior1st',
                       'Exterior2nd']].to_numpy())

        self.data.drop(['BsmtFinType1',
                        'BsmtFinType2',
                        'Exterior1st',
                        'Exterior2nd'], axis=1, inplace=True)


    def _encode_order(self):
        """
        Метод, объединяющий все кодирования порядковых переменных.
        """

        to_QC_encode = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                    'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual',
                    'GarageCond']

        self.data[to_QC_encode]       = self.data[to_QC_encode]\
                                        .applymap(self._QC_encoder)
        self.data['BsmtExposure']     =  self.data['BsmtExposure']\
                                        .apply(self._encode_exposure)
        self.data['GarageFinish']     =  self.data['GarageFinish']\
                                        .apply(self._encode_garage)



    def _fill_missing(self):
        """
        Метод, заменяющий пропуски в категориальных переменных на 'NO'
        (отсутствие чего-либо) и в вещественных переменных на 0
        (тоже встречается при отсутствии того или иного атрибута жилья).
        """

        self.data[self.categorical] = self.data[self.categorical].fillna('NO')
        self.data[self.numerical] = self.data[self.numerical].fillna(0)


    def _drop_features(self):
        """
        Удаление переменных, которые были признаны неважными в ходе EDA и
        кросс-валидации.
        """

        to_drop = ['PoolArea',
                   'GarageCond',
                   'MoSold',
                   'YrSold',
                   'GarageArea',
                   'TotRmsAbvGrd',
                   'MiscVal',
                   'BsmtFinSF2',
                   'Street',
                   'Utilities',
                   'RoofMatl',
                   'Condition2',
                   'Id',
                   'LandSlope',
                   'BsmtCond',
                   ]

        self.data.drop(to_drop, inplace=True, axis=1)

    def _year_compare(self, row: pd.Series, value: str) -> int:
        """
        Метод для бинаризации года постройки гаража/ремонта.

        return: 1, если позже года основания дома, 0 в ином случае
        """

        return int(row[value] > row.YearBuilt)


    def _remod_bin(self, row: pd.Series) -> int:
        """
        Метод для бинаризации года ремонта.

        return: 1, если гараж построен не в тот же год, что и дом, 0 иначе
        """

        return self._year_compare(row, 'YearRemodAdd')


    def _garage_bin(self, row: pd.Series) -> int:
        """
        Метод для бинаризации года ремонта.

        return: 1, если ремонта не было (стоит тот же год, что и год постройки дома),
        0 иначе
        """

        return self._year_compare(row, 'GarageYrBlt')


    def _binarize(self):
        """
        Метод, объединяющий перевод некоторых переменных в двоичный вид.
        """

        self.data[self.zero_threshold] =  self.data[self.zero_threshold]\
                                          .applymap(lambda x: int(x > 0))

        self.data[self.one_threshold] =  self.data[self.one_threshold]\
                                          .apply(lambda x: int(x == 1))

        self.data['Heating']         =  self.data['Heating']\
                                          .apply(lambda x: int(x == 'GasA'))

        self.data['YearRemodAdd'] = self.data.apply(self._remod_bin, axis=1)
        self.data['GarageYrBlt'] = self.data.apply(self._garage_bin, axis=1)


    def _sum_FlrSf(self):
        """
        Генерация признака, представляющего собой сумму площадей обоих этажей.
        """

        self.data['SumFlrSf'] = self.data['2ndFlrSF'] + self.data['1stFlrSF']
        self.data.drop(['2ndFlrSF', '1stFlrSF'], axis=1, inplace=True)


    def preprocess(self, encode_categories: bool=False):
        """
        Проведение этапов препроцессинга.

        Если выборка обучающая, то из нее убираются выбросы.
        """

        self._encode_order()
        self._fill_missing()
        self._binarize()
        self._multilabel()
        self._sum_FlrSf()

        self._drop_features()
        if self.is_train:
            self.data = self.data[self.data['LotArea'] < 50000]

In [ ]:
df = Data(df_train)
df.preprocess()

train = df.data
X = train.loc[:, train.columns != 'SalePrice']
y = train.SalePrice

test = Data(df_test, is_train=False)
test.preprocess()

test = test.data

In [ ]:
scaler = StandardScaler()

numeric = X.select_dtypes(exclude='object').columns
X[numeric] = scaler.fit_transform(X[numeric])
test[numeric] = scaler.transform(test[numeric])

<ipython-input-13-ce0f3b51fad7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric] = scaler.fit_transform(X[numeric])


###Обучение модели

In [ ]:
categorical_features_indices = np.where(X.dtypes != float)[0]

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

In [ ]:
!pip install catboost -q

In [ ]:
from catboost import CatBoostRegressor, Pool, metrics, cv
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
model = CatBoostRegressor(
    learning_rate=0.05,
    iterations=2000,
    max_depth=3,
    subsample=0.2,
    random_seed=42,
    logging_level='Silent'
)

In [ ]:
cv_params = model.get_params()

cv_data = cv(
    Pool(X_train, y_train, cat_features=categorical_features_indices),
    cv_params
)


In [ ]:
cv_data['test-RMSE-mean'][1995:]

1995    26175.554607
1996    26175.088700
1997    26173.985966
1998    26173.399440
1999    26173.632645
Name: test-RMSE-mean, dtype: float64

In [ ]:
model.fit(X_train, y_train,
          cat_features=categorical_features_indices,
          eval_set=(X_validation, y_validation),
          logging_level='Verbose')

0:	learn: 78466.1917059	test: 68772.9624984	best: 68772.9624984 (0)	total: 4.84ms	remaining: 9.68s
1:	learn: 76332.4708285	test: 66800.6351316	best: 66800.6351316 (1)	total: 9.96ms	remaining: 9.95s
2:	learn: 74056.0565130	test: 64644.1540695	best: 64644.1540695 (2)	total: 13.6ms	remaining: 9.07s
3:	learn: 71961.7097104	test: 62639.2280774	best: 62639.2280774 (3)	total: 17.8ms	remaining: 8.9s
4:	learn: 70135.3038801	test: 61092.7105585	best: 61092.7105585 (4)	total: 22.4ms	remaining: 8.95s
5:	learn: 68358.0572111	test: 59597.4833751	best: 59597.4833751 (5)	total: 26.4ms	remaining: 8.78s
6:	learn: 66549.0342039	test: 57921.0813340	best: 57921.0813340 (6)	total: 30.1ms	remaining: 8.56s
7:	learn: 64751.6901036	test: 56392.2654081	best: 56392.2654081 (7)	total: 34.1ms	remaining: 8.49s
8:	learn: 63009.0129974	test: 54733.7462848	best: 54733.7462848 (8)	total: 38.2ms	remaining: 8.44s
9:	learn: 61444.0759712	test: 53407.1323407	best: 53407.1323407 (9)	total: 42.4ms	remaining: 8.44s
10:	learn: 

In [ ]:
def log_rmse(pred, test):
    return MSE(np.log(1 + pred), np.log(1 + test), squared=False)


log_rmse(model.predict(X_validation), y_validation)

0.1214375878941708

In [ ]:
model.fit(X, y, cat_features=categorical_features_indices)

pd.DataFrame({'Id': df_test.Id, 'SalePrice': model.predict(test)}).to_csv('submission.csv', index=False)